# basic test for brib

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import rexpi
print("rexpi version = %s" % rexpi.__version__)

### define setup

In [ ]:
n=5
tol=1e-8
w = rexpi.west(n, tol)
wmax = (n+1)*np.pi
print("use n = %d and w = %f (max w \u2248%.2f). computed for error objective epsilon = %.2e"%(n,w,wmax,tol))

### compute best approximation

In [ ]:
r, info = rexpi.brib(w,n,info=1)
print("used %d iterations, error = %.2e, deviation = %.2e"%(info['iterations'],info['err'],info['dev']))

# generate various plots
### plot approximation error on [-1,1]

In [ ]:
xs = np.linspace(-1,1,3000)
err = r(1j*xs)-np.exp(1j*w*xs)
errmax = np.max(abs(err))
plt.semilogy(xs,abs(err),[-1,1],[errmax,errmax],':');
plt.ylabel("$|r(\mathrm{i}x)-\mathrm{e}^{\mathrm{i}\omega x}|$")
plt.xlabel("$x$");
plt.title("approximation error, $n=%d$, $\omega\\approx %.4f$"%(n,w));

### plot phase error on [-1,1]

In [ ]:
phaseerr = np.angle(r(1j*xs)/np.exp(1j*w*xs))
phaseerrmax = max(abs(phaseerr))
plt.plot([-1,1],[0,0],'k:',[-1,1],[phaseerrmax,phaseerrmax],'k:',[-1,1],[-phaseerrmax,-phaseerrmax],'k:')
plt.plot(xs,phaseerr);
plt.ylabel("$g(x)-\omega x$")
plt.xlabel("$x$");
plt.title("phase error, $n=%d$, $\omega\\approx%.4f$"%(n,w));
#plt.savefig("phaseerror.png")

### plot error path in complex plane
error at points of equioscillating phase error is numbered in ascending order

In [ ]:
pcirc=np.linspace(0,2*np.pi,100)
plt.plot(errmax*np.cos(pcirc),errmax*np.sin(pcirc),'k:')
plt.plot(np.real(err),np.imag(err));
for j,eta in enumerate(info['eta']):
    erreta = r(1j*eta)-np.exp(1j*w*eta)
    plt.text(erreta.real, erreta.imag, str(j+n+2), fontsize=12)
    plt.text(erreta.real, -erreta.imag, str(n+1-j), fontsize=12)
plt.title("path of error for $x\in[-1,1]$, $n=%d$, $\omega\\approx %.4f$"%(n,w))
plt.ylabel("Im $r(\mathrm{i} x)-\mathrm{e}^{\mathrm{i}\omega x}$")
plt.xlabel("Re $r(\mathrm{i} x)-\mathrm{e}^{\mathrm{i}\omega x}$");


### plot error in complex plane

In [ ]:
reftol = 10.0**(int(np.log10(tol))-1)
ntags = abs(int(np.log10(tol))-2)
tagslist = np.geomspace(reftol,1,ntags)
fmt = lambda x : "%.1e"%x

nim=400
nre=151
ximag = np.linspace(-1.4, 1.4, nim)
xreal = np.linspace(-2.0, 0.4, nre)
X, Y = np.meshgrid(xreal, ximag)
zall = (X+1j*Y).flatten()

# evaluate in barycentric rational form
rz = r(zall)
errorvec = abs(rz-np.exp(w*zall))
Z = np.reshape(errorvec,(nim,nre))

fig, ax = plt.subplots()

CS = ax.contour(X, Y, Z, levels=tagslist, locator=plt.LogLocator())
ax.clabel(CS, inline=1, fontsize=10, fmt=fmt)
plt.ylabel("Im $z$")
plt.xlabel("Re $z$");
plt.title("approximation error $\log_{10}|r(z)-\mathrm{e}^{\omega z}|$, $n=%d$, $\omega\\approx %.4f$"%(n,w));


### plot $|r(z)|$ in complex plane
stability properties imply $|r(z)|<1$ for $\text{Re}\,z < 0$

In [ ]:
nim=800
nre=300
ximag = np.linspace(-3.4, 3.4, nim)
xreal = np.linspace(-1.0, 0.2, nre)
X, Y = np.meshgrid(xreal, ximag)
zall = (X+1j*Y).flatten()

# evaluate in barycentric rational form
rz = r(zall)
absvec = abs(rz)
Zabsbary = np.reshape(absvec,(nim,nre))

# reference
absvec = abs(np.exp(w*zall))
Zabsref = np.reshape(absvec,(nim,nre))


tagslist = list(np.linspace(0.1,1.9,7))
fig, (ax1,ax2) = plt.subplots(2,1,figsize=(6, 6))

ax1.title.set_text('absolute value $|r(z)|$')
reftol = int(np.log10(tol))-1
CS = ax1.contour(X, Y, (Zabsbary), levels=tagslist)
ax1.clabel(CS, inline=1, fontsize=10)
ax1.set_ylabel("Im $z$")
ax1.axes.xaxis.set_ticklabels([])

ax2.title.set_text('reference $|\mathrm{e}^{\omega z}|$')
CS = plt.contour(X, Y, (Zabsref), levels=tagslist)
ax2.clabel(CS, inline=1, fontsize=10)
ax2.set_ylabel("Im $z$")
ax2.set_xlabel("Re $z$");


#plt.title("approximation error $\log_{10}|r(z)-\mathrm{e}^{\omega z}|$, $n=%d$, $\omega\\approx %.4f$"%(n,w));

import tikzplotlib
figname = "errinC"
figscaling = 1.3
tikzheight, tikzwidth = figscaling*6, figscaling*8
tikzplotlib.save(figname+".tex", axis_height='{:.2f}cm'.format(tikzheight), axis_width = '{:.2f}cm'.format(tikzwidth))


### plot poles in complex plane

In [ ]:
sj = r.getpoles(sym=True)
plt.plot(sj.real,sj.imag,'x')
plt.ylabel("Im $s_j$")
plt.xlabel("Re $s_j$");
plt.title("poles $s_j$ for $j=1,..,n$ with $n=%d$, $\omega\\approx %.4f$"%(n,w));